# Solving with start cache
Restraining the search space only to the values that can legaly be placed. Valid values are only calculated before search start and used through the whole search process.

In [1]:
#imports
import time
import copy

In [4]:
#Helper functionss

#to check if a proposed number is valid for the sudoku solution
def is_valid(sudoku, row, col, num):
    # Check row
    for x in range(9):
        if sudoku[row][x] == num: #checks if number is in row
            return False
    # Check column
    for x in range(9):
        if sudoku[x][col] == num: #checks if number is in col
            return False
    # Check 3x3 subgrid
    start_row = 3 * (row // 3)
    start_col = 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if sudoku[i + start_row][j + start_col] == num:
                return False
    return True

def find_empty_location(sudoku):
    for row in range(9):
        for col in range(9):
            if sudoku[row][col] == 0:
                return (row, col)
    return None

def create_cache(sudoku):
    valid_values = {}
    for row in range(9):
        for col in range(9):
            if sudoku[row][col] == 0:
                valid_values[(row, col)] = []
                for num in range(1, 10):
                    if is_valid(sudoku, row, col, num):
                        valid_values[(row, col)].append(num)
    return valid_values

def solve_sudoku(sudoku, cache):
    # print_sudoku(sudoku)
    # print()
    empty_location = find_empty_location(sudoku)
    if not empty_location:
        return True  # No empty cell left, puzzle solved
    row, col = empty_location

    # Only check values that are legaly allowed
    for num in cache[(row, col)]: 
        if is_valid(sudoku, row, col, num):  
            sudoku[row][col] = num  
            # print(f"Placing {num} at ({row}, {col})")
            if solve_sudoku(sudoku, cache): 
                return True
            # print(f"Backtracking from ({row}, {col}), removing {num}")
            sudoku[row][col] = 0
    return False


def print_sudoku(sudoku):
    for row in sudoku:
        print(row)

def time_solve(sudoku):
    result = copy.deepcopy(sudoku)
    start_time = time.time()
    cache = create_cache(result)
    if solve_sudoku(result, cache):
        print("Sudoku solved successfully!")
        print_sudoku(result)
    else:
        print("No solution exists.")
    print(f"Time taken: {time.time() - start_time:.7f} seconds")

In [6]:
# Datatype of sudoku
sudoku_grid = [
    [0, 0, 0, 0, 0, 0, 0, 1, 2],
    [0, 0, 0, 0, 3, 5, 0, 0, 0],
    [0, 0, 0, 4, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 7, 0, 0, 0],
    [0, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 6, 0, 0],
    [0, 7, 0, 9, 1, 0, 0, 0, 0],
    [5, 4, 0, 0, 0, 0, 0, 0, 0]
]

sudoku_hard = [
    [0, 0, 0, 7, 2, 4, 0, 0, 0],
    [1, 0, 0, 0, 9, 0, 0, 0, 0],
    [9, 0, 6, 0, 3, 0, 0, 0, 0],
    [5, 8, 0, 0, 0, 0, 0, 6, 0],
    [0, 0, 0, 0, 0, 7, 0, 3, 0],
    [0, 0, 0, 8, 0, 0, 9, 0, 0],
    [0, 0, 2, 3, 0, 0, 0, 0, 7],
    [0, 9, 0, 0, 0, 0, 4, 0, 2],
    [0, 0, 4, 0, 0, 0, 0, 0, 0]
]

time_solve(sudoku_grid)
time_solve(sudoku_hard)

Sudoku solved successfully!
[3, 5, 4, 6, 7, 8, 9, 1, 2]
[1, 6, 9, 2, 3, 5, 4, 7, 8]
[2, 8, 7, 4, 9, 1, 3, 5, 6]
[4, 1, 5, 3, 8, 2, 7, 6, 9]
[6, 9, 8, 1, 4, 7, 2, 3, 5]
[7, 3, 2, 5, 6, 9, 1, 8, 4]
[9, 2, 1, 8, 5, 3, 6, 4, 7]
[8, 7, 6, 9, 1, 4, 5, 2, 3]
[5, 4, 3, 7, 2, 6, 8, 9, 1]
Time taken: 0.0020525 seconds
Sudoku solved successfully!
[8, 3, 5, 7, 2, 4, 6, 1, 9]
[1, 2, 7, 6, 9, 8, 5, 4, 3]
[9, 4, 6, 5, 3, 1, 2, 7, 8]
[5, 8, 3, 9, 1, 2, 7, 6, 4]
[2, 6, 9, 4, 5, 7, 8, 3, 1]
[4, 7, 1, 8, 6, 3, 9, 2, 5]
[6, 5, 2, 3, 4, 9, 1, 8, 7]
[3, 9, 8, 1, 7, 6, 4, 5, 2]
[7, 1, 4, 2, 8, 5, 3, 9, 6]
Time taken: 0.0947583 seconds
